In [1]:
from itertools import combinations,product
from fractions import Fraction

In [2]:
def all_combinations(elements):
    answ = [[]]
    
    for i in range(1, len(elements)+1):
        answ.extend([list(x) for x in combinations(elements, i)])
    return answ

In [3]:
from enum import Enum
class EvalState(Enum):
    Nope = 0
    Evaluating = 1
    Evaluated = 2
    
class Position:
    def __init__(self, state_prob_pairs, name = 0):
        self.state_prob_pairs = state_prob_pairs
        self.eval_state = EvalState.Nope
        self.name = name
        
    def Q(self):
        if self.eval_state == EvalState.Nope:
            self.Evaluate()
        if self.eval_state != EvalState.Evaluated:
            raise Exception('Cyclic game')
        return self.q
        
    def Evaluate(self):
        self.eval_state = EvalState.Evaluating
        self.q = Fraction(1)
        for state, prob in self.state_prob_pairs:
            if len(state) > 0:
                self.q -= prob * min(state, key=lambda s: s.Q()).Q()
            else:
                self.q -= prob
        self.eval_state = EvalState.Evaluated

class Game:
    def __init__(self, positions, start):
        self.positions = positions
        self.start = start
        
    def Evaluate(self):
        self.start.Evaluate()     
        
    def Q(self):
        return self.start.Q()
    
    def __add__(self, game):
        positions = []
        for i in range(len(self.positions)):
            self.positions[i].num = i
        for i in range(len(game.positions)):
            game.positions[i].num = i
        for p2 in game.positions:
            for p1 in self.positions:
                positions.append(Position([], Game.ConcNames(p1.name, p2.name)))
        n = len(self.positions)
        for i in range(len(self.positions)):
            for j in range(len(game.positions)):
                for state1, prob1 in self.positions[i].state_prob_pairs:
                    for state2, prob2 in game.positions[j].state_prob_pairs:
                        state1_nums = list(map(lambda x: x.num, state1))
                        state2_nums = list(map(lambda x: x.num, state2))
                        new_state = list(map(lambda x: positions[x + j*n], state1_nums)) +\
                        list(map(lambda x: positions[i + x*n], state2_nums))
                        positions[i+n*j].state_prob_pairs.append((new_state, prob1 * prob2))
        return Game(positions, positions[self.start.num + n*game.start.num])
    
    @staticmethod
    def ConcNames(name1, name2):
        if (type(name1) != tuple):
            name1 = (name1,)
        if (type(name2) != tuple):
            name2 = (name2,)
        return name1 + name2
    
    def __str__(self):
        self.Evaluate()
        strs = []
        for p in self.positions:
            strs.append("{0}: {1}".format(p.name, p.Q()))
        return '\n'.join(strs) + '\n'

In [4]:
def NumberGame(n, p):
    positions = []
    positions.append(Position([([], Fraction(1))], 0))
    for i in range(1,n+1):
        state_prob_pairs = []
        for state in all_combinations(positions):
            state_prob_pairs.append((state, p**len(state) * (1-p)**(len(positions) - len(state))))
        positions.append(Position(state_prob_pairs, i))
    return Game(positions, positions[-1])

def DeterministicNumberGame(n):
    positions = []
    positions.append(Position([([], Fraction(1))], 0))
    for i in range(1,n+1):
        state_prob_pairs = []
        state_prob_pairs.append((positions.copy(), Fraction(1)))
        positions.append(Position(state_prob_pairs, i))
    return Game(positions, positions[-1])

In [5]:
g1 = NumberGame(2, Fraction(1,3))
g2 = NumberGame(3, Fraction(2,5))
g3 = g1 + g2
print(g1)
print(g2)
print(g3)

0: 0
1: 1/3
2: 13/27

0: 0
1: 2/5
2: 68/125
3: 9526/15625

(0, 0): 0
(1, 0): 1/3
(2, 0): 13/27
(0, 1): 2/5
(1, 1): 29/75
(2, 1): 2612/6075
(0, 2): 68/125
(1, 2): 4393/9375
(2, 2): 1722802/3796875
(0, 3): 9526/15625
(1, 3): 3038053/5859375
(2, 3): 28903091488/59326171875



In [6]:
print(NumberGame(1, Fraction(1,3)).Q())

1/3


In [7]:
#контрпример к аксиоме 6
fr = Fraction(1,3)
game1 = NumberGame(2, fr)
print(game1.Q())
positions = []
positions.append(Position([([], Fraction(1))], 0))
positions.append(Position([([], 1 - fr), ([positions[0]], fr)], 1))
positions.append(Position([([], (1 - fr)*(1 - fr)), ([positions[1]], fr*(1 - fr)), ([positions[0], positions[1]], fr)], 2))
game2 = Game(positions, positions[-1])
print(game2.Q())
game3 = NumberGame(1, Fraction(1,2))
print(game1)
print(game2)
print(game3)
print(game1 + game3)
print(game2 + game3)

13/27
13/27
0: 0
1: 1/3
2: 13/27

0: 0
1: 1/3
2: 13/27

0: 0
1: 1/2

(0, 0): 0
(1, 0): 1/3
(2, 0): 13/27
(0, 1): 1/2
(1, 1): 5/12
(2, 1): 137/324

(0, 0): 0
(1, 0): 1/3
(2, 0): 13/27
(0, 1): 1/2
(1, 1): 5/12
(2, 1): 427/972



In [8]:
#пример неизоморфных q-экв. игр (гипотеза 3)
fr = Fraction(1,3)
g1 = NumberGame(2, fr)
ps = []
ps.append(Position([([], Fraction(1))], 0))
ps.append(Position([([], 1 - fr), ([ps[0]], fr)], 1))
ps.append(Position([([], 1 - fr), ([ps[0]], fr)], 2))
ps.append(Position([([], (1 - fr)*(1 - fr)), 
                    ([ps[0]], fr*(1 - fr)), 
                    ([ps[1]], fr*(1 - fr)/2),
                    ([ps[2]], fr*(1 - fr)/2),
                    ([ps[0],ps[1]], fr*fr/2),
                    ([ps[0],ps[2]], fr*fr/2)], 3))
g2 = Game(ps, ps[-1])
g3 = NumberGame(3, Fraction(2,5))
print(g1.Q())
print(g2.Q())
print((g1+g3).Q())
print((g2+g3).Q())

13/27
13/27
28903091488/59326171875
28903091488/59326171875


In [9]:
#пример не q-эквивалентных игр с одинаковым q (гипотеза 2)
game1 = NumberGame(1, Fraction(13,27))
game2 = NumberGame(2, Fraction(1,3))
game3 = NumberGame(3, Fraction(1,3))

print(game1.Q())
print(game2.Q())
print((game1 + game3).Q())
print((game2 + game3).Q())

13/27
13/27
246905/531441
2237963/4782969
